In [2]:
## Load everything!

import numpy as np
import h5py
import time
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from scipy.optimize import curve_fit
from chillyinductor.rp22_helper import *
from colorama import Fore, Style
import sys
import os
import pandas as pd

#------------------------------------------------------------
# Import Data
datapath = get_datadir_path(rp=22, smc='A')
if datapath is None:
	print(f"{Fore.RED}Failed to find data location{Style.RESET_ALL}")
	sys.exit()
else:
	print(f"{Fore.GREEN}Located data directory at: {Fore.LIGHTBLACK_EX}{datapath}{Style.RESET_ALL}")
# filename = "dMS2_t3_04June2024_r1.hdf"
filename = "dMS2_t4_05June2024_r1.hdf"
# filename = "dMS2_t3_06June2024_5mA6_r1.hdf"

analysis_file = os.path.join(datapath, filename)

USE_CONST_SCALE = True

##--------------------------------------------
# Read HDF5 File

t_hdfr_0 = time.time()
with h5py.File(analysis_file, 'r') as fh:
	
	READ_MODE = 'dataset'
	df_cond_dict = {'powermeter_dBm': fh[READ_MODE]['coupled_power_meas_dBm'],
			 'rf_enabled': fh[READ_MODE]['rf_enabled'],
			 'lo_enabled': fh[READ_MODE]['lo_enabled'],
			 'freq_rf_GHz': fh[READ_MODE]['freq_rf_GHz'],
			 'freq_lo_GHz': fh[READ_MODE]['freq_lo_GHz'],
			 'power_rf_dBm': fh[READ_MODE]['power_RF_dBm'],
			 'power_lo_dBm': fh[READ_MODE]['power_LO_dBm']
			 }
	df_sa_dict = {'wav_f_Hz': list(fh[READ_MODE]['waveform_f_Hz']),
			 'wav_s_dBm': list(fh[READ_MODE]['waveform_s_dBm']),
			 'wav_rbw_Hz': list(fh[READ_MODE]['waveform_rbw_Hz']),
			 }
	
	READ_MODE = 'calibration_data'
	cf_cond_dict = {'powermeter_dBm': fh[READ_MODE]['coupled_power_meas_dBm'],
			 'rf_enabled': fh[READ_MODE]['rf_enabled'],
			 'lo_enabled': fh[READ_MODE]['lo_enabled'],
			 'freq_rf_GHz': fh[READ_MODE]['freq_rf_GHz'],
			 'freq_lo_GHz': fh[READ_MODE]['freq_lo_GHz'],
			 'power_rf_dBm': fh[READ_MODE]['power_RF_dBm'],
			 'power_lo_dBm': fh[READ_MODE]['power_LO_dBm']
			 }
	cf_sa_dict = {'wav_f_Hz': list(fh[READ_MODE]['waveform_f_Hz']),
			 'wav_s_dBm': list(fh[READ_MODE]['waveform_s_dBm']),
			 'wav_rbw_Hz': list(fh[READ_MODE]['waveform_rbw_Hz']),
			 }
	
	cf_cond = pd.DataFrame(cf_cond_dict)
	cf_sa = pd.DataFrame(cf_sa_dict)
	
	df_cond = pd.DataFrame(df_cond_dict)
	df_sa = pd.DataFrame(df_sa_dict)
	
	# Trim values greater than 10e3
	X = 10e3
	I_match = (df_cond <= X).all(axis=1)
	df_cond = df_cond[I_match]
	df_sa = df_sa[I_match]
	I_match = (cf_cond <= X).all(axis=1)
	cf_cond = cf_cond[I_match]
	cf_sa = cf_sa[I_match]
	
	print(f"{Fore.YELLOW}DF_COND:{Style.RESET_ALL}")
	print(df_cond)
	print(f"{Fore.YELLOW}DF_SA:{Style.RESET_ALL}")
	print(df_sa)

Located data directory at: E:\ARC0 PhD Data\RP-22 Lk Dil Fridge 2024\Data\SMC-A Downconversion v1
DF_COND:
    powermeter_dBm  rf_enabled  lo_enabled  freq_rf_GHz  freq_lo_GHz   
0        -18.28497        True        True          1.0          0.1  \
1        -17.87924        True        True          1.0          0.2   
2        -17.35060        True        True          1.0          0.3   
3        -16.75686        True        True          1.0          0.4   
4        -16.13419        True        True          1.0          0.5   
..             ...         ...         ...          ...          ...   
85       -12.55723        True        True          6.0          1.1   
86       -12.42567        True        True          6.0          1.2   
87       -12.53250        True        True          6.0          1.3   
88       -12.51764        True        True          6.0          1.4   
89       -12.36964        True        True          6.0          1.5   

    power_rf_dBm  power_lo_d

In [3]:
# Calculate mixing data DF
df_mix = calc_mixing_data(df_cond, df_sa)

### Plot surface graphs of the spectral components and their dependencies on LO and RF frequencies

In the cell below, the mx1l graph with datafile: "dMS2_t3_06June2024_5mA6_r1.hdf" shows a generally flat response, except for two lines that extend as a function of f_RF and f_LO, one of which is a strong peak, the other a strong trough. Perhaps this is some interesting resonant behavior.

In [5]:
## Make graphs of each power level

# Deploy ipython magic to get the graphs windowed rather than inline
%matplotlib auto

X,Y,Zmx1l = dfplot3d(df_mix, xparam='freq_rf_GHz', yparam='freq_lo_GHz', zparam='peak_mx1l', skip_plot=False, fig_no=1, show_markers=True, hovertips=False)
X,Y,Zrf1 = dfplot3d(df_mix, xparam='freq_rf_GHz', yparam='freq_lo_GHz', zparam='peak_rf1', skip_plot=False, fig_no=2, show_markers=True, hovertips=False)
X,Y,Zlo1 = dfplot3d(df_mix, xparam='freq_rf_GHz', yparam='freq_lo_GHz', zparam='peak_lo1', skip_plot=False, fig_no=3, show_markers=True, hovertips=False)

X,Y,Zmx1h = dfplot3d(df_mix, xparam='freq_rf_GHz', yparam='freq_lo_GHz', zparam='peak_mx1h', skip_plot=False, fig_no=4, show_markers=True, hovertips=False)
X,Y,Zmx2l = dfplot3d(df_mix, xparam='freq_rf_GHz', yparam='freq_lo_GHz', zparam='peak_mx2l', skip_plot=False, fig_no=5, show_markers=True, hovertips=False)
X,Y,Zmx2h = dfplot3d(df_mix, xparam='freq_rf_GHz', yparam='freq_lo_GHz', zparam='peak_mx2h', skip_plot=False, fig_no=6, show_markers=True, hovertips=False)

plt.show()

Using matplotlib backend: QtAgg


In [4]:
## Make graphs of mixing loss

lplot3d(X, Y, Zmx1l-Zrf1, xparam='freq_rf_GHz', yparam='freq_lo_GHz', zparam='mx1l-rf1', fig_no=7, show_markers=True, hovertips=False)

plt.show()

: 